In [26]:
!pip install pyswarms

In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from lmfit import minimize, Parameters, Parameter, report_fit
import pyswarms as ps

In [2]:
#Define constants
global r, K, Vc, nAHL
r1 = 1.2 #hr-1
K1 = 10**10  #CFU/mL
Vc = 10**(-12)  #mL Volume of one cell
nAHL = 1000

global t, data
t = np.array([0.0,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,5.5,6.0,6.5,7.0,7.5,8.0,8.5,9.0,9.5,10.0,10.5,11.0,11.5,12.0,12.5,
13.0,13.5,14.0,14.5,15.0,15.5,16.0,16.5,17.0,17.5,18.0,18.5,19.0,19.5,20.0,20.5,21.0,21.5,22.0,22.5,23.0,23.5,24.0,24.5,
25.0,25.5,26.0,26.5,27.0,27.5,28.0,28.5,29.0,29.5,30.0,30.5,31.0,31.5,32.0,32.5,33.0,33.5,34.0,34.5,35.0,35.5,36.0,36.5,37.0,
37.5,38.0,38.5,39.0,39.5,40.0,40.5,41.0,41.5,42.0,42.5,43.0,43.5,44.0,44.5,45.0,45.5,46.0,46.5,47.0,47.5])

data = np.array([474,509.3333333,657.3333333,748.6666667,889.6666667,1001.333333,1213,1468.666667,1639.666667,1834.333333,
2040.333333,2255.666667,2447.333333,2650,2839.666667,3053.666667,3218.333333,3386,3510,3596.333333,3634,3657,3651,3622,
3604.666667,3541.333333,3451,3359.666667,3259.666667,3149.666667,3055.666667,2949.333333,2860,2781.666667,2709.666667,
2647.666667,2610.666667,2560.333333,2506,2449,2396.333333,2338.333333,2270.333333,2214,2142.666667,2096.333333,2044,
1988.666667,1947.666667,1910.666667,1868,1843.333333,1810,1776.333333,1759,1725.666667,1708,1681,1654.666667,1627.333333,1607.333333,
1594.333333,1563.333333,1540.333333,1527.333333,1504.666667,1482.333333,1467.333333,1453,1428.333333,1410.666667,1397.666667,
1379.666667,1367.666667,1355.333333,1348,1334.333333,1321,1312.666667,1302,1282.333333,1280.666667,1261.666667,1244,1229.333333,
1214.333333,1198.333333,1188.333333,1172.333333,1154.333333,1148,1134.666667,1113.333333,1108.333333,1094.666667,1088.333333
])
                
data = np.reshape(data, (96,1))


In [3]:
#Function to generate ODEs for the qCRISPRi system

def ode_gen_stringent_qcrispri(y, t, p ):
    #unpack y values. 
    AHL, dCas9, GFP, N  = y
    
    alpha_pl = p['alpha_pl']
    alpha_rl = p['alpha_rl']
    alpha_ra = p['alpha_ra']
    alpha_g= p['alpha_g']
    
    nHill_A = p['nHill_A']
    nHill_c = p['nHill_c']
    
    Kd = p['Kd'] 
    KdCas = p['KdCas'] 
    
    Kdeg_A = p['Kdeg_A'] 
    Kdeg_Cas = p['Kdeg_Cas'] 
    Kdeg_G = p['Kdeg_G']
    m1 = p['m1']
    c1 = p['c1'] 
    
    
    values = [
              (alpha_pl + alpha_rl + (alpha_ra)*(AHL**nHill_A)/(Kd**nHill_A + AHL**nHill_A))*Vc*nHill_A*N - Kdeg_A*AHL, #AHL
              alpha_pl + alpha_rl + (alpha_ra)*(AHL**nHill_A)/(Kd**nHill_A + AHL**nHill_A) - Kdeg_Cas*dCas9, #dCas9
              m1*(alpha_g*0.1 + alpha_g*0.9*(KdCas**nHill_c)/(KdCas**nHill_c + dCas9**nHill_c)- Kdeg_G*GFP) + c1, #GFP
              r1*N*(1-N/K1) #N
            ]
    
    return(values)
  
def g(t, x0, p):
  """
  Solution to the ODE x'(t) = f(t,x,k) with initial condition x(0) = x0
  """
  sol = odeint(ode_gen_stringent_qcrispri, x0, t, args=(p,))
  return sol

def least_squares(gfp, data):
    return(sum([(i - j)**2 for i, j in zip(gfp, data)]))

def residual(particles):
    x0 = [0, 0, 1000, 10**7]
    obj_value = []
    for X in particles:
        p = {'alpha_pl': X[0],
             'alpha_rl': X[1],
             'alpha_ra': X[2],
             'alpha_g' : X[3],
             'Kd'      : X[4],
             'KdCas'   : X[5],
             'Kdeg_A'      : X[6],
             'Kdeg_Cas'    : X[7],
             'Kdeg_G'      : X[8],
             'm1'       : X[9],
             'c1'       : X[10],
             'nHill_A'      : X[11],
             'nHill_c'      : X[12]}
        model_sol = g(t, x0, p)
        GFP = np.reshape(model_sol[:, 2], (96,1))
        obj_value.append(least_squares(GFP, data)[0])
    return (obj_value)

In [4]:
# Set-up hyperparameters
options = { 'c1' : 2.8,  'c2' : 2,  'w' :0.9}

constraints = (np.array([0, 0 , 10 , 100 , 0,    0,  0, 0, 0, 0, 100 , 1,  0]),
               np.array([ 1, 1.5, 80 , 350 , 100,  3, 2, 4, 4,  5,1000, 2,  1]))


# Call instance of PSO
optimizer = ps.single.GlobalBestPSO(n_particles=20000, dimensions=13, options=options, bounds = constraints)

# Perform optimization
cost, pos = optimizer.optimize(residual, iters=50)

2023-10-20 21:49:22,393 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 2.8, 'c2': 2, 'w': 0.9}
pyswarms.single.global_best:   0%|                                        |0/50


KeyboardInterrupt: 

In [8]:
p = {'alpha_pl': pos[0],
     'alpha_rl': pos[1],
     'alpha_ra': pos[2],
     'alpha_g' : pos[3],
     'Kd'      : pos[4],
     'KdCas'   : pos[5],
     'Kdeg_A'  : pos[6],
     'Kdeg_Cas'    : pos[7],
     'Kdeg_G'      : pos[8],
     'm1'       : pos[9],
     'c1'       : pos[10],
     'nHill_A'      : pos[11],
     'nHill_c'      : pos[12]}
sim_gfp = g(t, [0, 0, 1000, 10**7], p)
plt.plot(t, data, 'o')
plt.plot(t, sim_gfp[:,2], '--', linewidth=2, c='blue');

In [ ]:
Final params used: [4.67441817e-01 7.36403506e-02 5.93643237e+01 2.38834527e+02
 3.64363621e+01 2.62536672e+00 1.90995169e+00 1.93552931e+00
 6.42944859e-02 4.64299180e+00 1.19480485e+02 1.02038175e+00
 9.85400641e-01]
best_cost=2.72e+6


new data:
    best cost: 19918502.190719567, best pos: [9.31141000e-03 1.28926261e+00 5.09873278e+01 2.43893318e+02
 1.74151165e+00 2.09443154e+00 7.33990053e-02 3.18903170e+00
 9.83017994e-02 3.71330439e+00 2.16070649e+02 1.91681377e+00
 9.02066447e-01]

In [ ]:
Kdeg_A = float(stringent_sys_params["Kdeg_A"])    #deg rate for AHL
Kdeg_Cas = float(stringent_sys_params["Kdeg_Cas"])     #deg rate for dCas9
Kdeg_G = float(stringent_sys_params["Kdeg_G"])      #deg rate for GFP
m1 = float(stringent_sys_params["m1"])     #
c1 = float(stringent_sys_params["c1"])     #
